# Little helpers for EDA

In [ ]:
import numpy as np
import json

%matplotlib inline

# missingno
> Messy datasets? Missing values? missingno provides a small toolset of flexible and easy-to-use missing data visualizations and utilities that allows you to get a quick visual summary of the completeness (or lack thereof) of your dataset.
> Just pip install missingno to get started.
> -- https://github.com/ResidentMario/missingno

In [ ]:
from quilt.data.ResidentMario import missingno_data


collisions = missingno_data.nyc_collision_factors()
collisions = collisions.replace("nan", np.nan)

In [ ]:
import missingno as msno


msno.matrix(collisions.sample(250))

# pandas_profiling
> Generates profile reports from a pandas DataFrame. The pandas df.describe() function is great but a little basic for serious exploratory data analysis.
> --https://github.com/pandas-profiling/pandas-profiling

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport

In [ ]:
df = pd.read_csv("data_v1.csv")

In [ ]:
report = ProfileReport(df)

In [ ]:
report.to_file("report_v1.html")

# great_expectations
> Great Expectations is a framework that helps teams save time and promote analytic integrity with a new twist on automated testing: pipeline tests. Pipeline tests are applied to data (instead of code) and at batch time (instead of compile or deploy time).
> -- https://github.com/great-expectations/great_expectations

In [ ]:
import great_expectations as ge

In [ ]:
df = ge.from_pandas(df)

type(df)

In [ ]:
df.head()

In [ ]:
df.Name.unique()

In [ ]:
df.expect_column_values_to_be_in_set(
    "Name",
    set(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']),
)

In [ ]:
df.expect_column_min_to_be_between(
    "PetalWidth",
    min_value=0.1,
)

In [ ]:
# there are many, many expectations!
# and you can write your own!
df.expec

In [ ]:
df.validate()

In [ ]:
df.save_expectations_config("expectations.json")

## Load and validate new data

In [ ]:
with open("expectations.json") as f:
    expectations = json.load(f)

df_v2 = ge.read_csv(
    "data_v1.csv",
    expectations_config=expectations,
)
df_v2.validate(only_return_failures=True)

In [ ]:
ge.validate(
    pd.read_csv("data_v1.csv"),
    expectations_config=expectations,
    only_return_failures=True,
)

## Load and validate "unexpected data"

In [ ]:
df = pd.read_csv("data_v1.csv")
df.loc[0, "PetalWidth"] = 0

In [ ]:
ge.validate(
    df,
    expectations_config=expectations,
    only_return_failures=True,
)